In [1]:
from distutils.command.install_egg_info import to_filename
import numpy as np
import streamlit as st
import cv2 
import tensorflow as tf
from tensorflow import keras
import torch

In [2]:
from transformers import AutoModelForImageClassification, AutoImageProcessor

# pretrain_path = r"C:\Users\User\Desktop\Code\Github\Final_project\WASSUP_EST_FINAL_Team4\swinv2-tiny-patch4-window8-256-finetuned-emotions\checkpoint-1509"
pretrain_path = r'C:\Users\User\Desktop\Code\Github\Final_project\swinv2-tiny-patch4-window8-256-finetuned-eurosat\checkpoint-2516'
model_trained = AutoModelForImageClassification.from_pretrained(pretrain_path)
test_img_processor = AutoImageProcessor.from_pretrained(pretrain_path)

In [3]:
from PIL import Image, ImageFont, ImageDraw



In [51]:
from transformers import pipeline

pipe = pipeline("image-classification", pretrain_path)

def img_with_text_results(img):
    
    #결과
    results = str(next(iter(pipe(Image.fromarray(img)))))
    
    # 폰트 색상 지정
    blue = (255, 0, 0)

    #한국 폰트
    kor_font = ImageFont.truetype('MaruBuri-Light.ttf', 20)
    img_pil = Image.fromarray(img)
    draw = ImageDraw.Draw(img_pil)
    draw.text((30,40), results, blue, font=kor_font)

    img = np.array(img_pil)
    # 폰트 지정
    # font = cv2.FONT_HERSHEY_PLAIN
    # 이미지에 글자 합성하기
    # img = cv2.putText(img, results, (30, 40), font, 2, blue, 1, cv2.LINE_AA)
    # img = cv2.putText(img, 'test', (100, 40), font, 2, blue, 1, cv2.LINE_AA)

    return img


In [52]:
# 비디오 캡처 객체 생성
cap = cv2.VideoCapture(0)

while True:
    
    # 프레임들 읽기
    ret, frame = cap.read()

    
    # 영상 처리 코드 작성
    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # for img in frame:
    #     enc = test_img_processor(img.convert("RGB"), return_tensors="pt")

    # enc = test_img_processor(frame, return_tensors="pt")
    
    # with torch.no_grad():
    #     outputs = model_trained(**enc)
    #     logits = outputs.logits
    #     predicted_class_idx = logits.argmax(-1).item()
        
        # predictions.append(model_trained.config.id2label[predicted_class_idx])
    # results = next(iter(pipe(frame)))

    img = img_with_text_results(frame)

    # 영상 출력
    
    cv2.imshow('Real-time Video', img)

    # 'q' 키를 누르면 종
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 객체 해제
cap.release()
cv2.destroyAllWindows()

In [ ]:
def video_input(data_src):
    vid_file = None
    if data_src == 'Sample data':
        vid_file = "data/sample_videos/sample.mp4"
    else:
        vid_bytes = st.sidebar.file_uploader("Upload a video", type=['mp4', 'mpv', 'avi'])
        if vid_bytes:
            vid_file = "data/uploaded_data/upload." + vid_bytes.name.split('.')[-1]
            with open(vid_file, 'wb') as out:
                out.write(vid_bytes.read())

    if vid_file:
        cap = cv2.VideoCapture(vid_file)
        custom_size = st.sidebar.checkbox("Custom frame size")
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        if custom_size:
            width = st.sidebar.number_input("Width", min_value=120, step=20, value=width)
            height = st.sidebar.number_input("Height", min_value=120, step=20, value=height)

        fps = 0
        st1, st2, st3 = st.columns(3)
        with st1:
            st.markdown("## Height")
            st1_text = st.markdown(f"{height}")
        with st2:
            st.markdown("## Width")
            st2_text = st.markdown(f"{width}")
        with st3:
            st.markdown("## FPS")
            st3_text = st.markdown(f"{fps}")

        st.markdown("---")
        output = st.empty()
        prev_time = 0
        curr_time = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                st.write("Can't read frame, stream ended? Exiting ....")
                break
            frame = cv2.resize(frame, (width, height))
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            output_img = infer_image(frame)
            output.image(output_img)
            curr_time = time.time()
            fps = 1 / (curr_time - prev_time)
            prev_time = curr_time
            st1_text.markdown(f"**{height}**")
            st2_text.markdown(f"**{width}**")
            st3_text.markdown(f"**{fps:.2f}**")

        cap.release()
